In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('hearthstone_standard_cards.csv')

In [4]:
df

,id,collectible,slug,classId,multiClassIds,cardTypeId,cardSetId,rarityId,artistName,manaCost,...,flavorText,cropImage,duels,minionTypeId,health,attack,keywordIds,childIds,durability,armor
0,58607,1,58607-blur,14,[],5,1463,1,Zoltan Boros,0,...,"""What are you trying to tell me? That I can do...",https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56806,1,56806-shadowhoof-slayer,14,[],4,2,2,A.J. Nazzaro,1,...,Was a humble Elven Archer before Sargeras got ...,https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",15.0,1.0,2.0,[8],NaN,NaN,NaN
2,58170,1,58170-crimson-sigil-runner,14,[],4,1414,1,Arthur Bozonnet,1,...,"He's not the fastest of the bunch, but he'll t...",https://d15f34w2p8l1cc.cloudfront.net/hearthst...,NaN,NaN,1.0,1.0,[86],NaN,NaN,NaN
3,59394,1,59394-demon-companion,14,"[14, 3]",5,1443,3,Zoltan Boros,1,...,Owning a pet can be real hell at times.,https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",NaN,NaN,NaN,NaN,"[59391, 59392, 59393]",NaN,NaN
4,59606,1,59606-double-jump,14,[],5,1443,1,A.J. Nazzaro,1,...,The key to every speed run.,https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",NaN,NaN,NaN,[86],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,61503,1,61503-cthun-the-shattered,12,[],4,1466,5,Alex Horley Orlandelli,10,...,My grandfather's deck has no useless cards!,https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",NaN,6.0,6.0,"[8, 64]","[61873, 61875, 61874, 61877]",NaN,NaN
1285,61629,1,61629-darkmoon-rabbit,12,[],4,1466,4,Matt Dixon,10,...,That's no ordinary rabbit! That's the most fou...,https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",20.0,1.0,1.0,"[32, 53]",NaN,NaN,NaN
1286,60443,1,60443-nzoth-god-of-the-deep,12,[],4,1466,5,Alex Horley Orlandelli,10,...,It really just wants to cuddle all the creatur...,https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",NaN,7.0,5.0,[8],NaN,NaN,NaN
1287,61308,1,61308-yshaarj-the-defiler,12,[],4,1466,5,Alex Horley Orlandelli,10,...,"Y'Shaarj made an excellent office intern, doin...",https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",NaN,10.0,10.0,"[8, 98]",[61296],NaN,NaN
